#### In this competition, you’re challenged to build an algorithm that automatically identifies if a remotely sensed target is a ship or iceberg. Improvements made will help drive the costs down for maintaining safe working conditions.

### MODEL 1

In [1]:
import numpy as np
import pandas as pd
import pylab
import cv2
import pdb
import scipy
import keras
import math
import tensorflow as tf

Using TensorFlow backend.


In [2]:
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom, map_coordinates
from scipy.ndimage import imread
from scipy.ndimage import rotate as rot
from scipy.ndimage.filters import gaussian_filter
from scipy.signal import convolve2d
from numpy.random import random_integers
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from skimage.restoration import (denoise_tv_chambolle, denoise_bilateral,
                                 denoise_wavelet, estimate_sigma,
                                 denoise_tv_bregman, denoise_nl_means)
from skimage.filters import gaussian
from skimage.color import rgb2gray
plt.rcParams['figure.figsize'] = 10,10
%matplotlib inline

In [3]:
from matplotlib import pyplot
from keras.utils.np_utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Activation
from keras.layers import GlobalMaxPooling2D, Lambda, Input, Flatten,LSTM
from keras.layers import ZeroPadding2D, GlobalAveragePooling2D, Merge
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.layers import *
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.models import Model, model_from_json, Sequential
from keras import initializers
from keras.optimizers import Adam, RMSprop, SGD, Adamax, Nadam, Adadelta, Adagrad
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.callbacks import ReduceLROnPlateau, History
from keras.datasets import cifar10
from keras import backend as K
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input

In [ ]:
train_1 = pd.read_json("train.json")

In [ ]:
test_1 = pd.read_json("test.json")

In [ ]:
# Generating the training bands
# create 3 bands having HH, HV and avg of both
X_band_1 = np.array([np.array(band).astype(np.float32).reshape(75,75) for band in train_1["band_1"]])
X_band_2 = np.array([np.array(band).astype(np.float32).reshape(75,75) for band in train_1["band_2"]])
X_train_1 = np.concatenate([X_band_1[:,:,:,np.newaxis], X_band_2[:,:,:,np.newaxis], ((X_band_1+X_band_2)/2)[:,:,:,np.newaxis]], axis =-1)

In [ ]:
def getModel_1():
    #Building the model
    gmodel = Sequential()
    
    #Conv Layer 1
    gmodel.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
    gmodel.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    gmodel.add(Dropout(0.3))
    
    #Conv Layer 2
    gmodel.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2,2), strides = (2,2)))
    gmodel.add(Dropout(0.3))
    
    #Conv Layer 3
    gmodel.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    gmodel.add(Dropout(0.3))
    
    #Conv Layer 4
    gmodel.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2,2), strides = (2,2)))
    gmodel.add(Dropout(0.3))
    
    #Flatten the data for upcoming dense layers
    gmodel.add(Flatten())
    
    #Dense Layer 1
    gmodel.add(Dense(512))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))
    
    #Dense Layer 2
    gmodel.add(Dense(256))
    gmodel.add(Activation('selu'))
    gmodel.add(Dropout(0.2))
    
    #Sigmoid layer
    gmodel.add(Dense(1))
    gmodel.add(Activation('sigmoid'))
    
    #mypotim = Adam(lr=0.001, decay=0.0)#lr=0.001
    mypotim = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-06, decay=0.0)
    
    gmodel.compile(loss = 'binary_crossentropy',
                   optimizer = mypotim,
                  metrics=['accuracy']
                  )
    gmodel.summary()
    return gmodel

def get_callbacks(filepath):
    es = EarlyStopping('val_loss', patience=10,mode='min')
    msave = ModelCheckpoint(filepath,save_best_only = True)
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss',
                                  factor = 0.1,
                                  patience=7,
                                  verbose = 0,
                                  epsilon=1e-4,
                                  mode='min')
    return [es,msave, reduce_lr_loss]

file_path = ".model_1_weights.hdf5"
callbacks = get_callbacks(filepath=file_path)

In [ ]:
target_train = train_1['is_iceberg']
X_train_cv, X_valid, y_train_cv ,y_valid = train_test_split(X_train_1,
                                                            target_train,
                                                            random_state=1,
                                                            train_size=0.75)

In [ ]:
#without denoising, core features
model_1 = getModel_1()
model_1.fit(X_train_cv,
           y_train_cv,
          batch_size =32,
          epochs=125,
          verbose=1,
          validation_data=(X_valid, y_valid),
          callbacks= callbacks
          )

In [ ]:
model_1.load_weights(filepath=file_path)
score_1 = model_1.evaluate(X_valid, y_valid, verbose = 1)
print('Train loss:', score_1[0])
print('Train accuracy:', score_1[1])

In [ ]:
X_band_test_1 = np.array([np.array(band).astype(np.float32).reshape(75,75) for band in test_1["band_1"]])
X_band_test_2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test_1["band_2"]])
X_test_1 = np.concatenate([X_band_test_1[:, :, :, np.newaxis], 
                         X_band_test_2[:, :, :, np.newaxis],
                         ((X_band_test_1+X_band_test_2)/2)[:, :, :, np.newaxis]], axis=-1)
predicted_test_1=model_1.predict_proba(X_test_1)

In [ ]:
submission_1 = pd.DataFrame()
submission_1['id']=test_1['id']
submission_1['is_iceberg']=predicted_test_1.reshape((predicted_test_1.shape[0]))
submission_1.to_csv('model_1.csv', index=False)

### MODEL 2

In [ ]:
train_2 = pd.read_json("train.json")

In [ ]:
def get_scaled_imgs(df):
    imgs = []
    
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75,75)
        band_2 = np.array(row['band_2']).reshape(75,75)
        band_3 = band_1 + band_2 
        
        #Rescale
        a = (band_1 - band_1.mean())/(band_1.max() - band_1.min())
        b = (band_2 - band_2.mean())/(band_2.max() - band_2.min())
        c = (band_3 - band_3.mean())/(band_3.max() - band_3.min())
        
        imgs.append(np.dstack((a,b,c)))
    return np.array(imgs)

In [ ]:
Xtrain_2 = get_scaled_imgs(train_2)
Ytrain_2 = np.array(train_2['is_iceberg'])

In [ ]:
train_2.inc_angle = train_2.inc_angle.replace('na',0)
idx_tr = np.where(train_2.inc_angle>0)

In [ ]:
Ytrain_2 = Ytrain_2[idx_tr[0]]
Xtrain_2 = Xtrain_2[idx_tr[0],...]

In [ ]:
def get_more_images(imgs):
    more_images = []
    vert_flip_imgs = []
    hori_flip_imgs = []
    
    for i in range(0,imgs.shape[0]):
        a = imgs[i,:,:,0]
        b = imgs[i,:,:,1]
        c = imgs[i,:,:,2]
        
        av = cv2.flip(a,1)
        ah = cv2.flip(a,0)
        bv = cv2.flip(b,1)
        bh = cv2.flip(b,0)
        cv = cv2.flip(c,1)
        ch = cv2.flip(c,0)
        
        vert_flip_imgs.append(np.dstack((av,bv,cv)))
        hori_flip_imgs.append(np.dstack((ah,bh,ch)))
        
    v = np.array(vert_flip_imgs)
    h = np.array(hori_flip_imgs)
    more_images = np.concatenate((imgs,v,h))
    
    return more_images

In [ ]:
Xtr_more_1 = get_more_images(Xtrain_2) 
Ytr_more_1 = np.concatenate((Ytrain_2,Ytrain_2,Ytrain_2))

In [ ]:
def getModel_2():
    model = Sequential()
    
    # CNN 1
    model.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 2
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 3
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    #CNN 4
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    # You must flatten the data for the dense layers
    model.add(Flatten())

    #Dense 1
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))

    #Dense 2
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))

    # Output 
    model.add(Dense(1, activation="sigmoid"))

    optimizer = Adam(lr=0.001, decay=0.0)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model
    

In [ ]:
model_2 = getModel_2()
batch_size = 32
earlyStopping = EarlyStopping(monitor = 'val_loss',
                              patience =10,
                              verbose = 0,
                              mode = 'min'
                             )
mcp_save = ModelCheckpoint('.model_2_weights.hdf5',
                           save_best_only=True,
                           monitor = 'val_loss',
                           mode = 'min'
                          )
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss',
                                  factor = 0.1,
                                  patience=7,
                                  verbose = 0,
                                  epsilon=1e-4,
                                  mode='min')

In [ ]:
model_2.fit(Xtr_more_1,
          Ytr_more_1,
          batch_size=batch_size,
          epochs=50,
          verbose=1,
         callbacks=[earlyStopping,mcp_save,reduce_lr_loss],
                       validation_split=0.25)

In [ ]:
model_2.load_weights(filepath = '.model_2_weights.hdf5')

score_2 = model_2.evaluate(Xtrain_2, Ytrain_2, verbose=1)
print('Train score:', score_2[0])
print('Train accuracy:', score_2[1])

In [ ]:
test_2 = pd.read_json('test.json')
test_2.inc_angle = test_2.inc_angle.replace('na',0)
Xtest_2 = (get_scaled_imgs(test_2))
pred_test_2 = model_2.predict(Xtest_2)

submission_2 = pd.DataFrame({'id': test_2["id"], 'is_iceberg': pred_test_2.reshape((pred_test_2.shape[0]))})
submission_2.to_csv('model_2.csv', index=False)

### MODEL 3

In [4]:
train_3 = pd.read_json("train.json")
test_3 = pd.read_json("test.json")
train_3.inc_angle = train_3.inc_angle.replace('na',0)
train_3.inc_angle = train_3.inc_angle.astype(float).fillna(0.0)

In [5]:
# Train data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train_3["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train_3["band_2"]])
X_train_3 = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]
                         , ((x_band1+x_band1)/2)[:, :, :, np.newaxis]], axis=-1)
X_angle_train_3 = np.array(train_3.inc_angle)
y_train_3 = np.array(train_3["is_iceberg"])

# Test data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test_3["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test_3["band_2"]])
X_test_3 = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]
                         , ((x_band1+x_band1)/2)[:, :, :, np.newaxis]], axis=-1)
X_angle_test_3 = np.array(test_3.inc_angle)


X_train_3, X_valid_3, X_angle_train_3, X_angle_valid_3, y_train_3, y_valid_3 = train_test_split(
    X_train_3, X_angle_train_3, y_train_3, random_state=123, train_size=0.75)

In [14]:
def get_callbacks(filepath):
    es = EarlyStopping('val_loss', patience=10, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss',
                                  factor = 0.1,
                                  patience=7,
                                  verbose = 0,
                                  epsilon=1e-4,
                                  mode='min')
    return [es, msave, reduce_lr_loss]
    
def get_model_3():
    bn_model = 0
    p_activation = "elu"
    
    input_1 = Input(shape=(75, 75, 3), name="X_1")
    input_2 = Input(shape=[1], name="angle")
    
    img_1 = Conv2D(16, kernel_size = (3,3), activation=p_activation) ((BatchNormalization(momentum=bn_model))(input_1))
    img_1 = Conv2D(16, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.2)(img_1)
    
    img_1 = Conv2D(32, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = Conv2D(32, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.2)(img_1)
    
    img_1 = Conv2D(64, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = Conv2D(64, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.2)(img_1)
    
    img_1 = Conv2D(128, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.2)(img_1)
    img_1 = GlobalMaxPooling2D() (img_1)
    
    img_2 = Conv2D(128, kernel_size = (3,3), activation=p_activation) ((BatchNormalization(momentum=bn_model))(input_1))
    img_2 = MaxPooling2D((2,2)) (img_2)
    img_2 = Dropout(0.2)(img_2)
    img_2 = GlobalMaxPooling2D() (img_2)
    
    img_concat =  (Concatenate()([img_1, img_2, BatchNormalization(momentum=bn_model)(input_2)]))
    
    dense_ayer = Dropout(0.5) (BatchNormalization(momentum=bn_model) ( Dense(256, activation=p_activation)(img_concat) ))
    dense_ayer = Dropout(0.5) (BatchNormalization(momentum=bn_model) ( Dense(64, activation=p_activation)(dense_ayer) ))
    output = Dense(1, activation="sigmoid")(dense_ayer)
    
    model = Model([input_1,input_2],  output)
    #optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)#0.8728
    optimizer = Adadelta(lr=1.0, rho=0.95, epsilon=1e-06, decay=0.0)#0.8952
    
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

In [15]:
file_path = ".model_3_weights.hdf5"
callbacks = get_callbacks(filepath=file_path)

model_3 = get_model_3()
model_3.fit([X_train_3,X_angle_train_3],
          y_train_3,
          epochs=50,
          validation_data=([X_valid_3, X_angle_valid_3], y_valid_3),
          batch_size=32,
          callbacks=callbacks
         )

Train on 1203 samples, validate on 401 samples
Epoch 1/50
1203/1203 [==============================] - 31s 26ms/step - loss: 0.8602 - acc: 0.5777 - val_loss: 0.8180 - val_acc: 0.5935
Epoch 2/50
1203/1203 [==============================] - 28s 23ms/step - loss: 0.7682 - acc: 0.5869 - val_loss: 0.6714 - val_acc: 0.5810
Epoch 3/50
1203/1203 [==============================] - 28s 23ms/step - loss: 0.6959 - acc: 0.5952 - val_loss: 1.2258 - val_acc: 0.5511
Epoch 4/50
1203/1203 [==============================] - 28s 24ms/step - loss: 0.6716 - acc: 0.6185 - val_loss: 0.6515 - val_acc: 0.5761
Epoch 5/50
1203/1203 [==============================] - 28s 24ms/step - loss: 0.6359 - acc: 0.6035 - val_loss: 1.1883 - val_acc: 0.5212
Epoch 6/50
1203/1203 [==============================] - 29s 24ms/step - loss: 0.6857 - acc: 0.6293 - val_loss: 0.8726 - val_acc: 0.6010
Epoch 7/50
1203/1203 [==============================] - 29s 24ms/step - loss: 0.6304 - acc: 0.6376 - val_loss: 0.6333 - val_acc: 0.6160
E

In [16]:
model_3.load_weights(filepath=file_path)

score_3 = (model_3.evaluate([X_valid_3, X_angle_valid_3], y_valid_3,verbose=1))

print('Train score:', score_3[0])
print('Train accuracy:', score_3[1])

401/401 [==============================] - 3s 7ms/step
Train score: 0.278252292675
Train accuracy: 0.885286783042


In [ ]:
prediction_3 = model_3.predict([X_test_3, X_angle_test_3],
                             verbose=1,
                             batch_size=200)

In [ ]:
submission_3 = pd.DataFrame({'id': test_3["id"], 'is_iceberg': prediction_3.reshape((prediction_3.shape[0]))})
submission_3.to_csv("model_3.csv", index=False)

### MODEL 4

In [4]:
train_4 = pd.read_json("train.json")

In [5]:
def get_images(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 / band_2

        # Rescale
        r = (band_1 - band_1.min()) / (band_1.max() - band_1.min())
        g = (band_2 - band_2.min()) / (band_2.max() - band_2.min())
        b = (band_3 - band_3.min()) / (band_3.max() - band_3.min())

        rgb = np.dstack((r, g, b))
        images.append(rgb)
    return np.array(images)

In [6]:
X = get_images(train_4)

In [7]:
y = to_categorical(train_4.is_iceberg.values,num_classes=2)

In [8]:
Xtr, Xv, ytr, yv = train_test_split(X, y, test_size=0.20)

In [9]:
def ConvBlock(model, layers, filters):
    '''Create [layers] layers consisting of zero padding, a convolution with [filters] 3x3 filters and batch normalization. Perform max pooling after the last layer.'''
    for i in range(layers):
        model.add(ZeroPadding2D((1, 1)))
        model.add(Conv2D(filters, (3, 3), activation='relu'))
        model.add(BatchNormalization(axis=3))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

In [10]:
def get_model_4():
    '''Create the FCN and return a keras model.'''

    model = Sequential()

    # Input image: 75x75x3
    model.add(Lambda(lambda x: x, input_shape=(75, 75, 3)))
    ConvBlock(model, 1, 64)
    # 37x37x32
    ConvBlock(model, 1, 128)
    # 18x18x64
    ConvBlock(model, 1, 128)
    # 9x9x128
    ConvBlock(model, 1, 64)
    # 4x4x128
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(2, (3, 3), activation='relu'))
    model.add(GlobalAveragePooling2D())
    
    # Output 
    model.add(Activation('sigmoid'))
    
    return model

In [11]:
model_4 = get_model_4()

myoptimizer = Adam(lr=0.0001, decay=0.0)

model_4.compile(loss='binary_crossentropy', optimizer=myoptimizer, metrics=['accuracy'])

In [12]:
datagen_4 = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen_4.fit(Xtr)

In [13]:
model_4.fit_generator(datagen_4.flow(Xtr,
                                     ytr,
                                     batch_size=32),
                      epochs=50,
                      validation_data =(Xv, yv),
                      verbose=1)

Epoch 1/50
41/41 [==============================] - 55s 1s/step - loss: 0.6517 - acc: 0.6062 - val_loss: 0.6902 - val_acc: 0.5000
Epoch 2/50
41/41 [==============================] - 54s 1s/step - loss: 0.6064 - acc: 0.6508 - val_loss: 0.6909 - val_acc: 0.5000
Epoch 3/50
41/41 [==============================] - 54s 1s/step - loss: 0.5997 - acc: 0.6623 - val_loss: 0.6967 - val_acc: 0.5000
Epoch 4/50
41/41 [==============================] - 51s 1s/step - loss: 0.5861 - acc: 0.6695 - val_loss: 0.7008 - val_acc: 0.5358
Epoch 5/50
41/41 [==============================] - 54s 1s/step - loss: 0.5737 - acc: 0.6861 - val_loss: 0.7348 - val_acc: 0.5498
Epoch 6/50
41/41 [==============================] - 53s 1s/step - loss: 0.5684 - acc: 0.6851 - val_loss: 0.7109 - val_acc: 0.5125
Epoch 7/50
41/41 [==============================] - 53s 1s/step - loss: 0.5500 - acc: 0.7070 - val_loss: 0.7294 - val_acc: 0.5358
Epoch 8/50
41/41 [==============================] - 53s 1s/step - loss: 0.5361 - acc: 0.70

In [15]:
model_4.save_weights('.model_4_weights.hdf5')

In [14]:
score_4 = (model_4.evaluate(Xv,yv,verbose=1))

print('Train score:', score_4[0])
print('Train accuracy:', score_4[1])

321/321 [==============================] - 5s 15ms/step
Train score: 0.50441255952
Train accuracy: 0.831775700935


In [ ]:
test_4 = pd.read_json('test.json')
Xtest_4 = get_images(test_4)
test_predictions_4 = model_4.predict_proba(Xtest_4)

In [ ]:
submission_4 = pd.DataFrame({'id': test_4['id'], 'is_iceberg': test_predictions_4[:, 1]})
submission_4.to_csv('model_4.csv', index=False)

### MODEL 5

In [ ]:
train_5 = pd.read_json("train.json")

In [ ]:
train_5.inc_angle = train_5.inc_angle.map(lambda x: 0.0 if x == 'na' else x)

In [ ]:
def transform (df):
    images = []
    for i, row in df.iterrows():
        band_1 = np.array(row['band_1']).reshape(75,75)
        band_2 = np.array(row['band_2']).reshape(75,75)
        band_3 = band_1 + band_2
        
        band_1_norm = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        band_2_norm = (band_2 - band_2. mean()) / (band_2.max() - band_2.min())
        band_3_norm = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())
        
        images.append(np.dstack((band_1_norm, band_2_norm, band_3_norm)))
    
    return np.array(images)

In [ ]:
def augment(images):
    image_mirror_lr = []
    image_mirror_ud = []
    image_rotate = []
    for i in range(0,images.shape[0]):
        band_1 = images[i,:,:,0]
        band_2 = images[i,:,:,1]
        band_3 = images[i,:,:,2]
            
        # mirror left-right
        band_1_mirror_lr = np.flip(band_1, 0)
        band_2_mirror_lr = np.flip(band_2, 0)
        band_3_mirror_lr = np.flip(band_3, 0)
        image_mirror_lr.append(np.dstack((band_1_mirror_lr, band_2_mirror_lr, band_3_mirror_lr)))
        
        # mirror up-down
        band_1_mirror_ud = np.flip(band_1, 1)
        band_2_mirror_ud = np.flip(band_2, 1)
        band_3_mirror_ud = np.flip(band_3, 1)
        image_mirror_ud.append(np.dstack((band_1_mirror_ud, band_2_mirror_ud, band_3_mirror_ud)))
        
        #rotate 
        band_1_rotate = rot(band_1, 30, reshape=False)
        band_2_rotate = rot(band_2, 30, reshape=False)
        band_3_rotate = rot(band_3, 30, reshape=False)
        image_rotate.append(np.dstack((band_1_rotate, band_2_rotate, band_3_rotate)))
        
    mirrorlr = np.array(image_mirror_lr)
    mirrorud = np.array(image_mirror_ud)
    rotated = np.array(image_rotate)
    images = np.concatenate((images, mirrorlr, mirrorud, rotated))
    return images

In [ ]:
train_X_5 = transform(train_5)
train_y_5 = np.array(train_5 ['is_iceberg'])

indx_tr = np.where(train_5.inc_angle > 0)

train_y_5 = train_y_5[indx_tr[0]]
train_X_5 = train_X_5[indx_tr[0], ...]

train_X_5 = augment(train_X_5)
train_y_5 = np.concatenate((train_y_5,train_y_5, train_y_5, train_y_5))

In [ ]:
def get_model_5():
    model = keras.models.Sequential()

    model.add(keras.layers.convolutional.Conv2D(64, kernel_size=(3,3), input_shape=(75,75,3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(keras.layers.convolutional.MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    model.add(keras.layers.Dropout(0.2))

    model.add(keras.layers.convolutional.Conv2D(128, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(keras.layers.convolutional.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(keras.layers.Dropout(0.2))

    model.add(keras.layers.convolutional.Conv2D(128, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(keras.layers.convolutional.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(keras.layers.Dropout(0.3))

    model.add(keras.layers.convolutional.Conv2D(64, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(keras.layers.convolutional.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(keras.layers.Dropout(0.3))

    model.add(keras.layers.Flatten())

    model.add(keras.layers.Dense(512))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(keras.layers.Dropout(0.2))

    model.add(keras.layers.Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(keras.layers.Dropout(0.2))


    model.add(keras.layers.Dense(1))
    model.add(Activation('sigmoid'))

    mypotim=Adam(lr=0.001, decay=0.0)
    
    model.compile(loss='binary_crossentropy', optimizer = mypotim, metrics=['accuracy'])

    return model

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='min')
mcp_save = ModelCheckpoint('.model_5_weights.hdf5',
                           save_best_only=True,
                           monitor='val_loss',
                           mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss',
                                   factor=0.1,
                                   patience=5,
                                   verbose=1,
                                   epsilon=1e-4,
                                   mode='min')

In [ ]:
model_5 = get_model_5()
model_5.fit(train_X_5,
            train_y_5,
            batch_size=32,
            epochs=50, 
            verbose=1,
            validation_split=0.25,
            callbacks=[early_stopping, reduce_lr_loss, mcp_save])

In [ ]:
score_5 = (model_5.evaluate(train_X_5,train_y_5,verbose=1))

print('Train score:', score_5[0])
print('Train accuracy:', score_5[1])

In [ ]:
test_5 = pd.read_json("test.json")
test_5.inc_angle = test_5.inc_angle.replace('na',0)
test_X_5 = transform(test_5)

In [ ]:
pred_test_5 = model_5.predict(test_X_5, verbose=1)
submission_5 = pd.DataFrame({'id': test_5["id"], 'is_iceberg': pred_test_5.reshape((pred_test_5.shape[0]))})
submission_5.to_csv('model_5.csv', index=False)

### MODEL 6

In [ ]:
train_6 = pd.read_json("train.json")

In [ ]:
def get_scaled_imgs(df):
    imgs = []
    
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75,75)
        band_2 = np.array(row['band_2']).reshape(75,75)
        band_3 = band_1 + band_2 
        
        #Rescale
        a = (band_1 - band_1.mean())/(band_1.max() - band_1.min())
        b = (band_2 - band_2.mean())/(band_2.max() - band_2.min())
        c = (band_3 - band_3.mean())/(band_3.max() - band_3.min())
        
        imgs.append(np.dstack((a,b,c)))
    return np.array(imgs)

In [ ]:
Xtrain_6 = get_scaled_imgs(train_6)
Ytrain_6 = np.array(train_6['is_iceberg'])

In [ ]:
train_6.inc_angle = train_6.inc_angle.replace('na',0)
idx_tr = np.where(train_6.inc_angle>0)

In [ ]:
Ytrain_6 = Ytrain_6[idx_tr[0]]
Xtrain_6 = Xtrain_6[idx_tr[0],...]

In [ ]:
def get_more_images(imgs):
    more_images = []
    vert_flip_imgs = []
    hori_flip_imgs = []
    
    for i in range(0,imgs.shape[0]):
        a = imgs[i,:,:,0]
        b = imgs[i,:,:,1]
        c = imgs[i,:,:,2]
        
        av = cv2.flip(a,1)
        ah = cv2.flip(a,0)
        bv = cv2.flip(b,1)
        bh = cv2.flip(b,0)
        cv = cv2.flip(c,1)
        ch = cv2.flip(c,0)
        
        vert_flip_imgs.append(np.dstack((av,bv,cv)))
        hori_flip_imgs.append(np.dstack((ah,bh,ch)))
        
    v = np.array(vert_flip_imgs)
    h = np.array(hori_flip_imgs)
    more_images = np.concatenate((imgs,v,h))
    
    return more_images

In [ ]:
Xtr_more_6 = get_more_images(Xtrain_6) 
Ytr_more_6 = np.concatenate((Ytrain_6,Ytrain_6,Ytrain_6))

In [ ]:
def getModel_6():
    model = Sequential()
    
    # CNN 1
    model.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 2
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 3
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    #CNN 4
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    # You must flatten the data for the dense layers
    model.add(Flatten())

    #Dense 1
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))

    #Dense 2
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))

    # Output 
    model.add(Dense(1, activation="sigmoid"))

    optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model
    

In [ ]:
model_6 = getModel_6()
earlyStopping = EarlyStopping(monitor = 'loss',
                              patience =10,
                              verbose = 0,
                              mode = 'min'
                             )
mcp_save = ModelCheckpoint('.model_6_weights.hdf5',
                           save_best_only=True,
                           save_weights_only=True,
                           monitor = 'loss',
                           mode = 'min'
                          )

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  # randomly flip images

datagen.fit(Xtr_more_6)

In [ ]:
model_6.fit_generator(datagen.flow(Xtr_more_6,
          Ytr_more_6,
          batch_size=32),
          epochs=50,
          verbose=1,
         callbacks=[earlyStopping,mcp_save])

In [ ]:
model_6.load_weights(filepath = '.model_6_weights.hdf5')

In [ ]:
#model_6.load_weights(filepath = '.model_2_weights.hdf5')#gave a 98% accuracy

score_6 = model_6.evaluate(Xtrain_6, Ytrain_6, verbose=1)
print('Train score:', score_6[0])
print('Train accuracy:', score_6[1])

In [ ]:
test_6 = pd.read_json('test.json')
test_6.inc_angle = test_6.inc_angle.replace('na',0)
Xtest_6 = (get_scaled_imgs(test_6))
pred_test_6 = model_6.predict(Xtest_6)

submission_6 = pd.DataFrame({'id': test_6["id"], 'is_iceberg': pred_test_6.reshape((pred_test_6.shape[0]))})
submission_6.to_csv('model_6.csv', index=False)

### MODEL 7

In [21]:
# Translate data to an image format
def color_composite(data):
    rgb_arrays = []
    for i, row in data.iterrows():
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 / band_2

        r = (band_1 + abs(band_1.min())) / np.max((band_1 + abs(band_1.min())))
        g = (band_2 + abs(band_2.min())) / np.max((band_2 + abs(band_2.min())))
        b = (band_3 + abs(band_3.min())) / np.max((band_3 + abs(band_3.min())))

        rgb = np.dstack((r, g, b))
        rgb_arrays.append(rgb)
    return np.array(rgb_arrays)

def denoise(X, weight, multichannel):
    return np.asarray([denoise_tv_chambolle(item, weight=weight, multichannel=multichannel) for item in X])

def smooth(X, sigma):
    return np.asarray([gaussian(item, sigma=sigma) for item in X])

def grayscale(X):
    return np.asarray([rgb2gray(item) for item in X])

In [22]:
train_7 = pd.read_json("train.json")
train_7.inc_angle = train_7.inc_angle.replace('na', 0)
train_7.inc_angle = train_7.inc_angle.astype(float).fillna(0.0)
train_all = True

# These are train flags that required to train model more efficiently and 
# select proper model parameters
train_b = True or train_all
train_img = True or train_all
train_total = True or train_all
predict_submission = True and train_all

clean_all = False
clean_b = False or clean_all
clean_img = False or clean_all

load_all = False
load_b = False or load_all
load_img = False or load_all

In [23]:
def create_dataset(frame, labeled, smooth_rgb=0.2, smooth_gray=0.5,
                   weight_rgb=0.05, weight_gray=0.05):
    band_1, band_2, images = frame['band_1'].values, frame['band_2'].values, color_composite(frame)
    to_arr = lambda x: np.asarray([np.asarray(item) for item in x])
    band_1 = to_arr(band_1)
    band_2 = to_arr(band_2)
    band_3 = (band_1 + band_2) / 2
    gray_reshape = lambda x: np.asarray([item.reshape(75, 75) for item in x])
    # Make a picture format from flat vector
    band_1 = gray_reshape(band_1)
    band_2 = gray_reshape(band_2)
    band_3 = gray_reshape(band_3)
    print('Denoising and reshaping')
    if train_b and clean_b:
        # Smooth and denoise data
        band_1 = smooth(denoise(band_1, weight_gray, False), smooth_gray)
        print('Gray 1 done')
        band_2 = smooth(denoise(band_2, weight_gray, False), smooth_gray)
        print('Gray 2 done')
        band_3 = smooth(denoise(band_3, weight_gray, False), smooth_gray)
        print('Gray 3 done')
    if train_img and clean_img:
        images = smooth(denoise(images, weight_rgb, True), smooth_rgb)
    print('RGB done')
    tf_reshape = lambda x: np.asarray([item.reshape(75, 75, 1) for item in x])
    band_1 = tf_reshape(band_1)
    band_2 = tf_reshape(band_2)
    band_3 = tf_reshape(band_3)
    #images = tf_reshape(images)
    band = np.concatenate([band_1, band_2, band_3], axis=3)
    if labeled:
        y = np.array(frame["is_iceberg"])
    else:
        y = None
    return y, band, images

In [32]:
y_train_7, X_b, X_images = create_dataset(train_7, True)

Denoising and reshaping
RGB done


In [25]:
def get_model_notebook(lr, decay, channels, relu_type='relu'):
    # angle variable defines if we should use angle parameter or ignore it
    input_1 = Input(shape=(75, 75, channels))

    fcnn = Conv2D(32, kernel_size=(3, 3), activation=relu_type)(BatchNormalization()(input_1))
    fcnn = MaxPooling2D((3, 3))(fcnn)
    fcnn = Dropout(0.2)(fcnn)
    fcnn = Conv2D(64, kernel_size=(3, 3), activation=relu_type)(fcnn)
    fcnn = MaxPooling2D((2, 2), strides=(2, 2))(fcnn)
    fcnn = Dropout(0.2)(fcnn)
    fcnn = Conv2D(128, kernel_size=(3, 3), activation=relu_type)(fcnn)
    fcnn = MaxPooling2D((2, 2), strides=(2, 2))(fcnn)
    fcnn = Dropout(0.2)(fcnn)
    fcnn = Conv2D(128, kernel_size=(3, 3), activation=relu_type)(fcnn)
    fcnn = MaxPooling2D((2, 2), strides=(2, 2))(fcnn)
    fcnn = Dropout(0.2)(fcnn)
    fcnn = BatchNormalization()(fcnn)
    fcnn = Flatten()(fcnn)
    local_input = input_1
    partial_model = Model(input_1, fcnn)
    dense = Dropout(0.2)(fcnn)
    dense = Dense(256, activation=relu_type)(dense)
    dense = Dropout(0.2)(dense)
    dense = Dense(128, activation=relu_type)(dense)
    dense = Dropout(0.2)(dense)
    dense = Dense(64, activation=relu_type)(dense)
    dense = Dropout(0.2)(dense)
    # For some reason i've decided not to normalize angle data
    output = Dense(1, activation="sigmoid")(dense)
    model = Model(local_input, output)
    optimizer = Adam(lr=lr, decay=decay)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model, partial_model

In [26]:
def combined_model(m_b, m_img, lr, decay):
    input_b = Input(shape=(75, 75, 3))
    input_img = Input(shape=(75, 75, 3))

    m1 = m_b(input_b)
    m2 = m_img(input_img)

    # So, combine models and train perceptron based on that
    # The iteresting idea is to use XGB for this task, but i actually hate this method
    common = Concatenate()([m1, m2])
    common = BatchNormalization()(common)
    common = Dropout(0.3)(common)
    common = Dense(1024, activation='relu')(common)
    common = Dropout(0.3)(common)
    common = Dense(512, activation='relu')(common)
    common = Dropout(0.3)(common)
    output = Dense(1, activation="sigmoid")(common)
    model = Model([input_b, input_img], output)
    optimizer = Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=decay)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

In [27]:
def gen_flow_multi_inputs(I1, I2, y, batch_size):
    gen1 = ImageDataGenerator(horizontal_flip=True,
                             vertical_flip=True,
                             width_shift_range=0.,
                             height_shift_range=0.,
                             channel_shift_range=0,
                             zoom_range=0.2,
                             rotation_range=10)
    gen2 = ImageDataGenerator(horizontal_flip=True,
                             vertical_flip=True,
                             width_shift_range=0.,
                             height_shift_range=0.,
                             channel_shift_range=0,
                             zoom_range=0.2,
                             rotation_range=10)
    genI1 = gen1.flow(I1, y, batch_size=batch_size, seed=57, shuffle=False)
    genI2 = gen2.flow(I1, I2, batch_size=batch_size, seed=57, shuffle=False)
    while True:
        I1i = genI1.next()
        I2i = genI2.next()
        #print I1i[0].shape
        np.testing.assert_array_equal(I2i[0], I1i[0])
        yield [I1i[0], I2i[1]], I1i[1]

In [29]:
def train_model(model, batch_size, epochs, checkpoint_name, X_train, y_train, val_data, verbose=2):
    callbacks = [ModelCheckpoint(checkpoint_name, save_best_only=True, monitor='val_loss')]
    datagen = ImageDataGenerator(horizontal_flip=True,
                                   vertical_flip=True,
                                   width_shift_range=0.,
                                   height_shift_range=0.,
                                   channel_shift_range=0,
                                   zoom_range=0.2,
                                   rotation_range=10)
    x_test, y_test = val_data
    try:
        model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), epochs=epochs,
                                    steps_per_epoch=len(X_train) / batch_size,
                                    validation_data=(x_test, y_test), verbose=1,
                                    callbacks=callbacks)
    except KeyboardInterrupt:
        if verbose > 0:
            print('Interrupted')
    if verbose > 0:
        print('Loading model')
    model.load_weights(filepath=checkpoint_name)
    return model

In [30]:
#Train a particular model
def gen_model_weights(lr, decay, channels, relu, batch_size, epochs, path_name, data, verbose=2):
    X_train, y_train, X_test, y_test, X_val, y_val = data
    model, partial_model = get_model_notebook(lr, decay, channels, relu)
    model = train_model(model, batch_size, epochs, path_name,
                           X_train, y_train, (X_test, y_test), verbose=verbose)

    if verbose > 0:
        loss_val, acc_val = model.evaluate(X_val, y_val,
                               verbose=0, batch_size=batch_size)

        loss_train, acc_train = model.evaluate(X_test, y_test,
                                       verbose=0, batch_size=batch_size)

        print('Val/Train Loss:', str(loss_val) + '/' + str(loss_train), \
            'Val/Train Acc:', str(acc_val) + '/' + str(acc_train))
    return model, partial_model

In [31]:
# Train all 3 models
def train_models(dataset, lr, batch_size, max_epoch, verbose=2, return_model=False):
    y_train, X_b, X_images = dataset
    y_train_full, y_val,\
    X_b_full, X_b_val,\
    X_images_full, X_images_val = train_test_split(y_train, X_b, X_images, random_state=687, train_size=0.9)

    y_train, y_test, \
    X_b_train, X_b_test, \
    X_images_train, X_images_test = train_test_split(y_train_full, X_b_full, X_images_full, random_state=576, train_size=0.85)

    if train_b:
        if verbose > 0:
            print('Training bandwidth network')
        data_b1 = (X_b_train, y_train, X_b_test, y_test, X_b_val, y_val)
        model_b, model_b_cut = gen_model_weights(lr, 1e-6, 3, 'relu', batch_size, max_epoch, 'model_b',
                                                 data=data_b1, verbose=verbose)

    if train_img:
        if verbose > 0:
            print('Training image network')
        data_images = (X_images_train, y_train, X_images_test, y_test, X_images_val, y_val)
        model_images, model_images_cut = gen_model_weights(lr, 1e-6, 3, 'relu', batch_size, max_epoch, 'model_img',
                                                       data_images, verbose=verbose)

    if train_total:
        common_model = combined_model(model_b_cut, model_images_cut, lr/2, 1e-7)
        common_x_train = [X_b_full, X_images_full]
        common_y_train = y_train_full
        common_x_val = [X_b_val, X_images_val]
        common_y_val = y_val
        if verbose > 0:
            print('Training common network')
        callbacks = [ModelCheckpoint('common', save_best_only=True, monitor='val_loss')]
        try:
            common_model.fit_generator(gen_flow_multi_inputs(X_b_full, X_images_full, y_train_full, batch_size),
                                         epochs=30,
                                  steps_per_epoch=len(X_b_full) / batch_size,
                                  validation_data=(common_x_val, common_y_val), verbose=1,
                                  callbacks=callbacks)
        except KeyboardInterrupt:
            pass
        common_model.load_weights(filepath='common')
        loss_val, acc_val = common_model.evaluate(common_x_val, common_y_val,
                                           verbose=0, batch_size=batch_size)
        loss_train, acc_train = common_model.evaluate(common_x_train, common_y_train,
                                                  verbose=0, batch_size=batch_size)
        if verbose > 0:
            print('Loss:', loss_val, 'Acc:', acc_val)
    if return_model:
        return common_model
    else:
        return (loss_train, acc_train), (loss_val, acc_val)

In [35]:
model_7 = train_models((y_train_7, X_b, X_images),
                       lr = 8e-5,
                       batch_size = 32,
                       max_epoch = 250,
                       verbose=1,
                       return_model=True)

Training bandwidth network
Epoch 1/250
39/38 [==============================] - 11s 282ms/step - loss: 0.7152 - acc: 0.5298 - val_loss: 0.6803 - val_acc: 0.5253
Epoch 2/250
39/38 [==============================] - 8s 211ms/step - loss: 0.6669 - acc: 0.5857 - val_loss: 0.6692 - val_acc: 0.6912
Epoch 3/250
39/38 [==============================] - 8s 206ms/step - loss: 0.6553 - acc: 0.5797 - val_loss: 0.6662 - val_acc: 0.6129
Epoch 4/250
39/38 [==============================] - 8s 211ms/step - loss: 0.6463 - acc: 0.5820 - val_loss: 0.6710 - val_acc: 0.5991
Epoch 5/250
39/38 [==============================] - 8s 210ms/step - loss: 0.6539 - acc: 0.5946 - val_loss: 0.6835 - val_acc: 0.5576
Epoch 6/250
39/38 [==============================] - 8s 200ms/step - loss: 0.6382 - acc: 0.6148 - val_loss: 0.6900 - val_acc: 0.5253
Epoch 7/250
39/38 [==============================] - 8s 202ms/step - loss: 0.6325 - acc: 0.6001 - val_loss: 0.7026 - val_acc: 0.4839
Epoch 8/250
39/38 [======================

Epoch 62/250
39/38 [==============================] - 9s 227ms/step - loss: 0.3689 - acc: 0.8257 - val_loss: 0.5670 - val_acc: 0.7235
Epoch 63/250
39/38 [==============================] - 9s 230ms/step - loss: 0.3820 - acc: 0.8169 - val_loss: 0.6385 - val_acc: 0.7005
Epoch 64/250
39/38 [==============================] - 8s 216ms/step - loss: 0.3513 - acc: 0.8372 - val_loss: 0.5880 - val_acc: 0.7143
Epoch 65/250
39/38 [==============================] - 8s 218ms/step - loss: 0.3568 - acc: 0.8329 - val_loss: 0.5674 - val_acc: 0.7373
Epoch 66/250
39/38 [==============================] - 8s 216ms/step - loss: 0.3570 - acc: 0.8445 - val_loss: 0.5130 - val_acc: 0.7880
Epoch 67/250
39/38 [==============================] - 9s 224ms/step - loss: 0.3578 - acc: 0.8347 - val_loss: 0.5536 - val_acc: 0.7419
Epoch 68/250
39/38 [==============================] - 10s 244ms/step - loss: 0.3475 - acc: 0.8425 - val_loss: 0.4983 - val_acc: 0.7558
Epoch 69/250
39/38 [==============================] - 9s 231m

Epoch 123/250
39/38 [==============================] - 8s 215ms/step - loss: 0.3067 - acc: 0.8677 - val_loss: 0.5877 - val_acc: 0.7465
Epoch 124/250
39/38 [==============================] - 8s 211ms/step - loss: 0.3208 - acc: 0.8565 - val_loss: 0.4927 - val_acc: 0.7926
Epoch 125/250
39/38 [==============================] - 9s 220ms/step - loss: 0.3117 - acc: 0.8516 - val_loss: 0.4240 - val_acc: 0.8157
Epoch 126/250
39/38 [==============================] - 8s 215ms/step - loss: 0.3121 - acc: 0.8749 - val_loss: 0.4890 - val_acc: 0.7926
Epoch 127/250
39/38 [==============================] - 9s 220ms/step - loss: 0.3063 - acc: 0.8612 - val_loss: 0.5681 - val_acc: 0.7696
Epoch 128/250
39/38 [==============================] - 8s 214ms/step - loss: 0.3062 - acc: 0.8636 - val_loss: 0.5701 - val_acc: 0.7742
Epoch 129/250
39/38 [==============================] - 9s 221ms/step - loss: 0.3149 - acc: 0.8649 - val_loss: 0.4726 - val_acc: 0.7972
Epoch 130/250
39/38 [==============================] - 

Epoch 184/250
39/38 [==============================] - 8s 212ms/step - loss: 0.3108 - acc: 0.8690 - val_loss: 0.5340 - val_acc: 0.7880
Epoch 185/250
39/38 [==============================] - 8s 206ms/step - loss: 0.2950 - acc: 0.8699 - val_loss: 0.4568 - val_acc: 0.8249
Epoch 186/250
39/38 [==============================] - 8s 212ms/step - loss: 0.2932 - acc: 0.8749 - val_loss: 0.5285 - val_acc: 0.7650
Epoch 187/250
39/38 [==============================] - 8s 205ms/step - loss: 0.2610 - acc: 0.8772 - val_loss: 0.4939 - val_acc: 0.8018
Epoch 188/250
39/38 [==============================] - 8s 211ms/step - loss: 0.2660 - acc: 0.8893 - val_loss: 0.5249 - val_acc: 0.8018
Epoch 189/250
39/38 [==============================] - 8s 206ms/step - loss: 0.2821 - acc: 0.8746 - val_loss: 0.4803 - val_acc: 0.8065
Epoch 190/250
39/38 [==============================] - 8s 212ms/step - loss: 0.2698 - acc: 0.8782 - val_loss: 0.4754 - val_acc: 0.8018
Epoch 191/250
39/38 [==============================] - 

Epoch 245/250
39/38 [==============================] - 9s 218ms/step - loss: 0.2417 - acc: 0.8902 - val_loss: 0.3754 - val_acc: 0.8618
Epoch 246/250
39/38 [==============================] - 8s 214ms/step - loss: 0.2711 - acc: 0.8801 - val_loss: 0.4022 - val_acc: 0.8525
Epoch 247/250
39/38 [==============================] - 9s 221ms/step - loss: 0.2712 - acc: 0.8875 - val_loss: 0.4430 - val_acc: 0.8387
Epoch 248/250
39/38 [==============================] - 8s 209ms/step - loss: 0.2404 - acc: 0.8997 - val_loss: 0.4503 - val_acc: 0.8387
Epoch 249/250
39/38 [==============================] - 8s 210ms/step - loss: 0.2469 - acc: 0.8998 - val_loss: 0.4514 - val_acc: 0.8249
Epoch 250/250
39/38 [==============================] - 8s 205ms/step - loss: 0.2996 - acc: 0.8713 - val_loss: 0.4049 - val_acc: 0.8664
Loading model
Val/Train Loss: 0.328736434627/0.375361672332 Val/Train Acc: 0.875776397516/0.861751151524
Training image network
Epoch 1/250
39/38 [==============================] - 11s 285ms

39/38 [==============================] - 7s 178ms/step - loss: 0.3171 - acc: 0.8540 - val_loss: 0.3360 - val_acc: 0.8157
Epoch 117/250
39/38 [==============================] - 7s 181ms/step - loss: 0.3408 - acc: 0.8402 - val_loss: 0.3385 - val_acc: 0.8203
Epoch 118/250
39/38 [==============================] - 7s 178ms/step - loss: 0.3534 - acc: 0.8323 - val_loss: 0.3470 - val_acc: 0.8295
Epoch 119/250
39/38 [==============================] - 7s 182ms/step - loss: 0.3086 - acc: 0.8636 - val_loss: 0.3237 - val_acc: 0.8295
Epoch 120/250
39/38 [==============================] - 7s 179ms/step - loss: 0.3147 - acc: 0.8620 - val_loss: 0.3344 - val_acc: 0.8387
Epoch 121/250
39/38 [==============================] - 7s 181ms/step - loss: 0.3117 - acc: 0.8449 - val_loss: 0.3455 - val_acc: 0.8157
Epoch 122/250
39/38 [==============================] - 7s 178ms/step - loss: 0.3157 - acc: 0.8557 - val_loss: 0.3386 - val_acc: 0.8433
Epoch 123/250
39/38 [==============================] - 7s 178ms/step 

Epoch 177/250
39/38 [==============================] - 7s 179ms/step - loss: 0.2940 - acc: 0.8603 - val_loss: 0.3449 - val_acc: 0.8111
Epoch 178/250
39/38 [==============================] - 7s 180ms/step - loss: 0.3130 - acc: 0.8520 - val_loss: 0.4137 - val_acc: 0.8065
Epoch 179/250
39/38 [==============================] - 7s 179ms/step - loss: 0.2893 - acc: 0.8596 - val_loss: 0.3256 - val_acc: 0.8203
Epoch 180/250
39/38 [==============================] - 7s 179ms/step - loss: 0.2900 - acc: 0.8725 - val_loss: 0.3268 - val_acc: 0.8203
Epoch 181/250
39/38 [==============================] - 7s 180ms/step - loss: 0.3016 - acc: 0.8612 - val_loss: 0.3472 - val_acc: 0.8111
Epoch 182/250
39/38 [==============================] - 7s 179ms/step - loss: 0.3021 - acc: 0.8587 - val_loss: 0.3284 - val_acc: 0.8341
Epoch 183/250
39/38 [==============================] - 7s 181ms/step - loss: 0.2934 - acc: 0.8538 - val_loss: 0.3849 - val_acc: 0.8065
Epoch 184/250
39/38 [==============================] - 

Epoch 238/250
39/38 [==============================] - 7s 180ms/step - loss: 0.2710 - acc: 0.8780 - val_loss: 0.3358 - val_acc: 0.8295
Epoch 239/250
39/38 [==============================] - 7s 178ms/step - loss: 0.2396 - acc: 0.8830 - val_loss: 0.3343 - val_acc: 0.8387
Epoch 240/250
39/38 [==============================] - 7s 180ms/step - loss: 0.2616 - acc: 0.8802 - val_loss: 0.3279 - val_acc: 0.8387
Epoch 241/250
39/38 [==============================] - 7s 177ms/step - loss: 0.2618 - acc: 0.8682 - val_loss: 0.3389 - val_acc: 0.8203
Epoch 242/250
39/38 [==============================] - 7s 181ms/step - loss: 0.2736 - acc: 0.8754 - val_loss: 0.3595 - val_acc: 0.8157
Epoch 243/250
39/38 [==============================] - 7s 178ms/step - loss: 0.2642 - acc: 0.8862 - val_loss: 0.3436 - val_acc: 0.8111
Epoch 244/250
39/38 [==============================] - 7s 182ms/step - loss: 0.2627 - acc: 0.8804 - val_loss: 0.3492 - val_acc: 0.8203
Epoch 245/250
39/38 [==============================] - 

In [37]:
if predict_submission:
    print('Reading test dataset')
    test_7 = pd.read_json("test.json")
    y_fin, X_fin_b, X_fin_img = create_dataset(test_7, False)
    print('Predicting')
    prediction_7 = model_7.predict([X_fin_b, X_fin_img], verbose=1, batch_size=32)
    print('Submitting')
    submission_7 = pd.DataFrame({'id': test_7["id"], 'is_iceberg': prediction_7.reshape((prediction_7.shape[0]))})

    submission_7.to_csv("model_7.csv", index=False)
    print('Done')

Reading test dataset
Denoising and reshaping
RGB done
Predicting
8424/8424 [==============================] - 37s 4ms/step
Submitting
Done


### STACK 1

In [44]:
sub_model_1 = pd.read_csv("model_1.csv", index_col=0)
sub_model_2 = pd.read_csv("model_2.csv", index_col=0)
sub_model_3 = pd.read_csv("model_3.csv", index_col=0)
#sub_model_4 = pd.read_csv("model_4.csv", index_col=0) #4b without 0.1430
sub_model_5 = pd.read_csv("model_5.csv", index_col=0)
sub_model_6 = pd.read_csv("model_6.csv", index_col=0)
sub_model_7 = pd.read_csv("model_7.csv", index_col=0) #4c with 0.1491

concat_sub = pd.concat([sub_model_1,sub_model_2,sub_model_3,sub_model_5,sub_model_6,sub_model_7], axis=1)
cols = list(map(lambda x: "is_iceberg_" + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)

In [ ]:
concat_sub.head()

In [ ]:
concat_sub.corr()

In [45]:
# get the data fields ready for stacking
concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 1:6].max(axis=1)
concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 1:6].min(axis=1)
concat_sub['is_iceberg_mean'] = concat_sub.iloc[:, 1:6].mean(axis=1)
concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 1:6].median(axis=1)

In [46]:
# set up cutoff threshold for lower and upper bounds, easy to twist 
cutoff_lo = 0.8
cutoff_hi = 0.2

In [ ]:
#Mean Stacking
concat_sub['is_iceberg'] = concat_sub['is_iceberg_mean']
concat_sub[['id', 'is_iceberg']].to_csv('stack_1.csv', 
                                        index=False, float_format='%.6f')

### STACK 2

In [ ]:
#Median Stacking
concat_sub['is_iceberg'] = concat_sub['is_iceberg_median']
concat_sub[['id', 'is_iceberg']].to_csv('stack_2.csv', 
                                        index=False, float_format='%.6f')

### STACK 3

In [ ]:
#Min max + Mean Stacking
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_mean']))
concat_sub[['id', 'is_iceberg']].to_csv('stack_3.csv', 
                                        index=False, float_format='%.6f')

### STACK 4

In [47]:
#Min max + Median Stacking
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_median']))
concat_sub[['id', 'is_iceberg']].to_csv('stack_4c.csv', 
                                        index=False, float_format='%.6f')

### STACK 5

In [ ]:
# load the model with best base performance
sub_base = pd.read_csv("model_2.csv")

In [ ]:
concat_sub['is_iceberg_base'] = sub_base['is_iceberg']
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_base']))
concat_sub[['id', 'is_iceberg']].to_csv('stack_5.csv', 
                                        index=False, float_format='%.6f')